In [1]:
## pulling from mongo db
from pymongo import MongoClient

In [2]:
hashtagToPull = 'wakenbake'

In [3]:
client = MongoClient('mongodb://localhost:27017/')
db = client['instagram']
collectionName = hashtagToPull + '-tags'
collection = db[collectionName]

In [4]:
media = collection.find() ## pull entire document store

In [5]:
## map only categories we need
mediaArr = []
for item in media:
    record = {}
    record['id'] = item['id']
    record['username'] = item['username']
    record['type'] = item['type']
    record['filter'] = item['filter']
    record['like_count'] = item['like_count']
    record['comment_count'] = item['comment_count']
    record['created_time'] = item['created_time']
    
    record['location.lat'] = ""
    record['location.lon'] = ""
    if len(item['location']) > 0:
        record['location.lat'] = item['location']['latitude']
        record['location.lon'] = item['location']['longitude']
    
    
    mediaArr.append(record)

In [6]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
df = pd.DataFrame(mediaArr)

In [8]:
df.head()

,comment_count,created_time,filter,id,like_count,location.lat,location.lon,type,username
0,1,2015-08-14 18:56:14,Normal,1051583761891237133_1621796574,3,,,image,mystical_stoner
1,1,2015-08-14 18:55:17,Normal,1051583276660595965_1621796574,6,,,image,mystical_stoner
2,0,2015-08-14 18:54:44,Reyes,1051583000928241682_1147602737,0,,,image,misscontroless1
3,1,2015-08-14 18:54:06,Normal,1051582683891162307_519339662,8,,,image,saulsilver_
4,1,2015-08-14 18:50:53,Crema,1051581066892725208_1147602737,0,,,image,misscontroless1


In [9]:
## count location tags
count = 0
for item in mediaArr:
    if item['location.lat'] != '':
        count+=1
        
print (count)

5650
